## Load the data:

In [1]:
import warnings
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt

# Suppress specific UserWarnings from openpyxl
warnings.filterwarnings("ignore", category=UserWarning, module='openpyxl')

# Define the directory where your files are located
data_dir = '.'  # Adjust the path according to your file location

# List to hold the dataframes
dataframes = []

# Loop through the years and load the files
for year in range(2000, 2020):
    if year <= 2012:
        file_path = os.path.join(data_dir, f'{year}.xls')
    else:
        file_path = os.path.join(data_dir, f'{year}.xlsx')
    
    # Load the file into a dataframe
    df = pd.read_excel(file_path)
    
    # Append the dataframe to the list
    dataframes.append(df)

# Concatenate all the dataframes into one
betting_data = pd.concat(dataframes, ignore_index=True)

# Ensure all columns are displayed
pd.set_option('display.max_columns', None)
# Ensure all columns are displayed
pd.set_option('display.max_columns', None)


## Fixing Anomalies

In [2]:
def is_column_numeric(df, column_name):
    # Check if the column contains only numeric values
    return df[column_name].apply(lambda x: str(x).isnumeric()).all()

# Check if columns are numeric before converting
anomaly_column = ['WRank', 'LRank', 'EXW']
for column in anomaly_column:
    if is_column_numeric(betting_data, column):
        print(f"Column '{column}' is numeric.\n")
    else:
        print(f"Column '{column}' is not numeric.\n")

def find_non_numeric_values(df, column_name):
    # Function to check if a value is numeric
    def is_numeric(value):
        try:
            float(value)
            return True
        except ValueError:
            return False

    # Apply the function to the column and filter non-numeric values
    non_numeric_values = df[~df[column_name].apply(is_numeric)]

    # Display the non-numeric values
    print(f"Non-numeric values in {column_name}:")
    print(non_numeric_values[[column_name]])

# WRank column
find_non_numeric_values(betting_data, 'WRank')

# LRank column
find_non_numeric_values(betting_data, 'LRank')

# EXW column
find_non_numeric_values(betting_data, 'EXW')

Column 'WRank' is not numeric.

Column 'LRank' is not numeric.

Column 'EXW' is not numeric.

Non-numeric values in WRank:
    WRank
744    NR
Non-numeric values in LRank:
     LRank
63      NR
377     NR
560     NR
611     NR
613     NR
618     NR
774     NR
1039    NR
1694    NR
1849    NR
3540    NR
3551    NR
4457    NR
5435    NR
5487    NR
5491    NR
5494    NR
5498    NR
5528    NR
5998    NR
6307    NR
6777    NR
7296    NR
7414    NR
7418    NR
8111    NR
9596    NR
Non-numeric values in EXW:
        EXW
38294  2.,3


In [3]:
# Convert WRank and LRank to numeric, coercing errors
betting_data['WRank'] = pd.to_numeric(betting_data['WRank'], errors='coerce')
betting_data['LRank'] = pd.to_numeric(betting_data['LRank'], errors='coerce')

# Fill NaN values with a high number
betting_data['WRank'].fillna(100000, inplace=True)
betting_data['LRank'].fillna(100000, inplace=True)


# Correct the typo in row 38294, column 'EXW'
if betting_data.at[38294, 'EXW'] == '2.,3':
    betting_data.at[38294, 'EXW'] = '2.3'


## Feature Engineering:

In [4]:
# Now perform the calculations
betting_data['higher_rank_won'] = (betting_data['WRank'] < betting_data['LRank']).astype(int)
betting_data['higher_rank_points'] = betting_data['higher_rank_won'] * betting_data['WPts'] + betting_data['LPts'] * (1 - betting_data['higher_rank_won'])
betting_data['lower_rank_points'] = (1 - betting_data['higher_rank_won']) * betting_data['WPts'] + betting_data['LPts'] * betting_data['higher_rank_won']
betting_data['higher_rank_points'].fillna(0, inplace=True)
betting_data['lower_rank_points'].fillna(0, inplace=True)

all_matches_k = betting_data.copy()

# Columns to drop
columns_to_drop = [
    'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets', 'Lsets', 'Comment', 
    'CBW', 'CBL', 'GBW', 'GBL', 'IWW', 'IWL', 'SBW', 'SBL', 'B365W', 'B365L', 'B&WW', 'B&WL', 
    'EXW', 'EXL', 'PSW', 'PSL', 'WPts', 'LPts', 'UBW', 'UBL', 'LBW', 'LBL', 'SJW', 'SJL', 
    'MaxW', 'MaxL', 'AvgW', 'AvgL'
]

# Drop the columns
all_matches_k = all_matches_k.drop(columns=columns_to_drop)

### K-factor model

In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss

# Initialize Elo ratings
initial_elo = 1500
players_elo = {}

def win_probability(E_i, E_j):
    return 1 / (1 + 10 ** ((E_j - E_i) / 400))

def get_player_elo(player_name):
    if player_name not in players_elo:
        players_elo[player_name] = initial_elo
    return players_elo[player_name]

def set_player_elo(player_name, elo):
    players_elo[player_name] = elo

def update_elo_k(E_i, E_j, K, outcome):
    pi_j = win_probability(E_i, E_j)  # Calculate win probability for player i against player j
    if outcome == 'win':
        delta_E_i = K * (1 - pi_j)  # Elo rating change for a win
        new_E_i = E_i + delta_E_i
    elif outcome == 'loss':
        delta_E_i = K * (pi_j - 1)  # Elo rating change for a loss
        new_E_i = E_j + delta_E_i
    else:
        raise ValueError("Outcome must be 'win' or 'loss'")
    return new_E_i

def evaluate_model(df):
    logloss_value = log_loss(df.higher_rank_won, df.prob_high_ranked)
    accuracy_value = np.mean(df.match_outcome == df.higher_rank_won)
    calibration_value = np.sum(df.prob_high_ranked) / np.sum(df.higher_rank_won)
    return logloss_value, accuracy_value, calibration_value

def update_elo_and_probabilities_k(df, K):
    for index, match in df.iterrows():
        winner_name, loser_name = match['Winner'], match['Loser']

        # Retrieve current Elo ratings
        winner_elo = get_player_elo(winner_name)
        loser_elo = get_player_elo(loser_name)
        
        # Calculate win probabilities
        df.at[index, 'prob_winner'] = win_probability(winner_elo, loser_elo)

        # Determine match outcomes based on probability and who was expected to win
        if match['higher_rank_won']:
            df.at[index, 'match_outcome'] = int(df.at[index, 'prob_winner'] > 0.5)
            df.at[index, 'prob_high_ranked'] = df.at[index, 'prob_winner']
        else:
            df.at[index, 'match_outcome'] = int((1 - df.at[index, 'prob_winner']) > 0.5)
            df.at[index, 'prob_high_ranked'] = 1 - df.at[index, 'prob_winner']

        # Update Elo ratings
        new_winner_elo = update_elo_k(winner_elo, loser_elo, K, 'win')
        new_loser_elo = update_elo_k(winner_elo, loser_elo, K, 'loss')
        set_player_elo(winner_name, new_winner_elo)
        set_player_elo(loser_name, new_loser_elo)


In [6]:
results = []

# Check K values from 1 to 100
for K in range(1, 101):
    players_elo = {}
    all_matches_k_elo = all_matches_k.copy()
    update_elo_and_probabilities_k(all_matches_k_elo, K)
    all_matches_k_elo['Date'] = pd.to_datetime(all_matches_k_elo['Date'], format='%Y-%m-%d')
    split_time = pd.to_datetime('2019-01-01', format='%Y-%m-%d')
    all_matches_k_elo_train = all_matches_k_elo[all_matches_k_elo['Date'] < split_time]
    all_matches_k_elo_validation = all_matches_k_elo[all_matches_k_elo['Date'] >= split_time]
    logloss_value, accuracy_value, calibration_value = evaluate_model(all_matches_k_elo_validation)
    results.append({
        'K': K,
        'logloss': logloss_value,
        'accuracy': accuracy_value,
        'calibration': calibration_value
    })

results_df = pd.DataFrame(results)
best_params_logloss = results_df.loc[results_df['logloss'].idxmin()]
best_params_accuracy = results_df.loc[results_df['accuracy'].idxmax()]
print(f"Best parameters based on log-loss: \n{best_params_logloss}\n")
print(f"Best parameters based on accuracy: \n{best_params_accuracy}\n")

# Prompt user for choice
choice = input("Update Elo ratings based on (logloss/accuracy): ").strip().lower()

if choice == "logloss":
    best_params = best_params_logloss
elif choice == "accuracy":
    best_params = best_params_accuracy
else:
    raise ValueError("Invalid choice! Please choose 'logloss' or 'accuracy'.")

print(f"Selected best parameters: \n{best_params}")

players_elo = {}
update_elo_and_probabilities_k(all_matches_k, best_params['K'])

# Optionally, update DataFrame with new Elo scores
all_matches_k['winner_new_elo'] = all_matches_k['Winner'].apply(get_player_elo)
all_matches_k['loser_new_elo'] = all_matches_k['Loser'].apply(get_player_elo)

all_matches_k

Best parameters based on log-loss: 
K              22.000000
logloss         0.634729
accuracy        0.630929
calibration     1.032152
Name: 21, dtype: float64

Best parameters based on accuracy: 
K              52.000000
logloss         0.645589
accuracy        0.640956
calibration     1.060109
Name: 51, dtype: float64

Update Elo ratings based on (logloss/accuracy): logloss
Selected best parameters: 
K              22.000000
logloss         0.634729
accuracy        0.630929
calibration     1.032152
Name: 21, dtype: float64


,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,higher_rank_won,higher_rank_points,lower_rank_points,prob_winner,match_outcome,prob_high_ranked,winner_new_elo,loser_new_elo
0,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Dosedel S.,Ljubicic I.,63.0,77.0,1,0.0,0.0,0.500000,0.0,0.500000,1426.441872,1669.643050
1,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Enqvist T.,Clement A.,5.0,56.0,1,0.0,0.0,0.500000,0.0,0.500000,1506.775739,1529.395476
2,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Escude N.,Baccanello P.,40.0,655.0,1,0.0,0.0,0.500000,0.0,0.500000,1678.038927,1468.167867
3,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Federer R.,Knippschild J.,65.0,87.0,1,0.0,0.0,0.500000,0.0,0.500000,2072.262466,1432.749935
4,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Fromberg R.,Woodbridge T.,81.0,198.0,1,0.0,0.0,0.500000,0.0,0.500000,1420.825964,1469.699553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54903,66,London,Masters Cup,2019-11-15,Masters Cup,Indoor,Hard,Round Robin,3,Nadal R.,Tsitsipas S.,1.0,6.0,1,9585.0,4000.0,0.468382,0.0,0.468382,1511.695603,1533.354477
54904,66,London,Masters Cup,2019-11-15,Masters Cup,Indoor,Hard,Round Robin,3,Zverev A.,Medvedev D.,7.0,4.0,0,5705.0,2945.0,0.531618,0.0,0.468382,1499.367493,1467.695603
54905,66,London,Masters Cup,2019-11-16,Masters Cup,Indoor,Hard,Semifinals,3,Tsitsipas S.,Federer R.,6.0,3.0,0,6190.0,4000.0,0.453576,1.0,0.546424,1533.354477,1530.634613
54906,66,London,Masters Cup,2019-11-16,Masters Cup,Indoor,Hard,Semifinals,3,Thiem D.,Zverev A.,5.0,7.0,1,5025.0,2945.0,0.502868,1.0,0.502868,1512.205432,1499.367493


## Split Dataset:

In [7]:
all_matches_k['Date'] = pd.to_datetime(all_matches_k['Date'], format='%Y-%m-%d')
split_time = pd.to_datetime('2019-01-01', format='%Y-%m-%d')
all_matches_k_train = all_matches_k[all_matches_k['Date'] < split_time]
all_matches_k_validation = all_matches_k[all_matches_k['Date'] >= split_time]
all_matches_k_train


,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,higher_rank_won,higher_rank_points,lower_rank_points,prob_winner,match_outcome,prob_high_ranked,winner_new_elo,loser_new_elo
0,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Dosedel S.,Ljubicic I.,63.0,77.0,1,0.0,0.0,0.500000,0.0,0.500000,1426.441872,1669.643050
1,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Enqvist T.,Clement A.,5.0,56.0,1,0.0,0.0,0.500000,0.0,0.500000,1506.775739,1529.395476
2,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Escude N.,Baccanello P.,40.0,655.0,1,0.0,0.0,0.500000,0.0,0.500000,1678.038927,1468.167867
3,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Federer R.,Knippschild J.,65.0,87.0,1,0.0,0.0,0.500000,0.0,0.500000,2072.262466,1432.749935
4,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Fromberg R.,Woodbridge T.,81.0,198.0,1,0.0,0.0,0.500000,0.0,0.500000,1420.825964,1469.699553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52356,3,Pune,Maharashtra Open,2018-12-31,ATP250,Outdoor,Hard,1st Round,3,Darcis S.,Carballes Baena R.,100000.0,73.0,0,0.0,0.0,0.606522,0.0,0.393478,1524.615616,1573.596625
52357,3,Pune,Maharashtra Open,2018-12-31,ATP250,Outdoor,Hard,1st Round,3,Munar J.,Albot R.,81.0,98.0,1,663.0,592.0,0.562198,1.0,0.562198,1522.977538,1633.590236
52358,3,Pune,Maharashtra Open,2018-12-31,ATP250,Outdoor,Hard,1st Round,3,Donskoy E.,Andujar P.,97.0,82.0,0,658.0,594.0,0.424818,1.0,0.575182,1500.909361,1581.441838
52359,3,Pune,Maharashtra Open,2018-12-31,ATP250,Outdoor,Hard,1st Round,3,Mmoh M.,Gunneswaran P.,103.0,110.0,1,563.0,521.0,0.517362,1.0,0.517362,1484.930946,1469.355382


In [8]:
all_matches_k_validation


,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,higher_rank_won,higher_rank_points,lower_rank_points,prob_winner,match_outcome,prob_high_ranked,winner_new_elo,loser_new_elo
52303,1,Brisbane,Brisbane International,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Kudla D.,Fritz T.,63.0,49.0,0,974.0,810.0,0.370555,1.0,0.629445,1512.259723,1668.272759
52304,1,Brisbane,Brisbane International,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Chardy J.,Struff J.L.,40.0,57.0,1,1050.0,875.0,0.468765,0.0,0.468765,1689.424753,1703.951998
52305,1,Brisbane,Brisbane International,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Murray A.,Duckworth J.,240.0,234.0,0,206.0,200.0,0.940442,0.0,0.059558,1890.789202,1474.042302
52306,1,Brisbane,Brisbane International,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Kyrgios N.,Harrison R.,35.0,62.0,1,1125.0,810.0,0.746424,1.0,0.746424,1775.247884,1577.037693
52307,1,Brisbane,Brisbane International,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Tsonga J.W.,Kokkinakis T.,239.0,146.0,0,367.0,200.0,0.797780,0.0,0.202220,1797.779333,1557.102093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54903,66,London,Masters Cup,2019-11-15,Masters Cup,Indoor,Hard,Round Robin,3,Nadal R.,Tsitsipas S.,1.0,6.0,1,9585.0,4000.0,0.468382,0.0,0.468382,1511.695603,1533.354477
54904,66,London,Masters Cup,2019-11-15,Masters Cup,Indoor,Hard,Round Robin,3,Zverev A.,Medvedev D.,7.0,4.0,0,5705.0,2945.0,0.531618,0.0,0.468382,1499.367493,1467.695603
54905,66,London,Masters Cup,2019-11-16,Masters Cup,Indoor,Hard,Semifinals,3,Tsitsipas S.,Federer R.,6.0,3.0,0,6190.0,4000.0,0.453576,1.0,0.546424,1533.354477,1530.634613
54906,66,London,Masters Cup,2019-11-16,Masters Cup,Indoor,Hard,Semifinals,3,Thiem D.,Zverev A.,5.0,7.0,1,5025.0,2945.0,0.502868,1.0,0.502868,1512.205432,1499.367493


## Evaluate Model Performance
#### Accuracy:  
Calculate the accuracy of your model predictions.

In [9]:
accuracy_k = np.mean(all_matches_k_validation.match_outcome == all_matches_k_validation.higher_rank_won)
print(f'Accuracy: {accuracy_k}')


Accuracy: 0.6309294253760124


#### Calibration:  
Assess the calibration of your model.

In [10]:
calibration_k = np.sum(all_matches_k_validation.prob_high_ranked)/np.sum(all_matches_k_validation.higher_rank_won)
print(f'Calibration: {calibration_k}')


Calibration: 1.0321522181571612


##### Log-loss:
Compute the log-loss for your predictions.

In [11]:
def logloss(actual, predictions):
    epsilon = 1e-15
    predictions = np.clip(predictions, epsilon, 1 - epsilon)
    
    logr_logloss_all_predictors = -(1 / len(actual)) * np.sum(
        actual * np.log(predictions) + (1 - actual) * np.log(1 - predictions))
    return logr_logloss_all_predictors


In [12]:
logloss_k = logloss(all_matches_k_validation.higher_rank_won, all_matches_k_validation.prob_high_ranked)
print(f'Logloss: {logloss_k}')


Logloss: 0.6347288809626205


In [22]:
# Create a DataFrame with the selected metrics
metric_comparison = pd.DataFrame({
    'model': ['K_Constant_25', 'K_Constant_Optimised'],
    'pred_acc': [results_df.iloc[24][2], accuracy_k],
    'log_loss': [results_df.iloc[24][1], logloss_k],
    'calibration': [results_df.iloc[24][3], calibration_k]
})

# Print the DataFrame
print(metric_comparison)


                  model  pred_acc  log_loss  calibration
0         K_Constant_25  0.630929  0.634947     1.037693
1  K_Constant_Optimised  0.630929  0.634729     1.032152
